In [9]:
import numpy as np
import pandas as pd
from keras import Model
import keras.backend as K
from keras.layers import Embedding,Reshape,Input,Dot
import keras
from keras.utils import np_utils 

In [10]:
'''
- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings
'''
rating_data=pd.read_csv("./ml-1m/ratings.dat",header=None,names=["UserID","MovieID","Rating","Timestamp"],sep='::')
rating_data=rating_data.sample(frac=1)
#len(rating_data)
traindata=rating_data[:int(len(rating_data)*8/10)]
validdata=rating_data[int(len(rating_data)*8/10):int(len(rating_data)*9/10)]
testdata=rating_data[int(len(rating_data)*9/10):]
print(len(traindata),len(validdata),len(testdata))

C:\Users\30249\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


800167 100021 100021


In [11]:
num_user = np.max(traindata["UserID"])
num_movie = np.max(traindata["MovieID"])
print(num_user,num_movie,len(traindata))

6040 3952 800167


In [127]:
K.clear_session()
def Recmand_model(num_user,num_movie,k):
    input_uer = Input(shape=[None,],dtype="int32")
    print(input_uer)
    model_uer = Embedding(num_user+1,k,input_length = 1)(input_uer)
    model_uer = Reshape((k,))(model_uer)
    
    input_movie = Input(shape=[None,],dtype="int32")
    model_movie  = Embedding(num_movie+1,k,input_length = 1)(input_movie)
    model_movie = Reshape((k,))(model_movie)
    
    out = Dot(1)([model_uer,model_movie])
    out2 = keras.layers.core.Dense(units=6,input_shape=(1,),kernel_initializer='normal',activation='softmax')(out)
    model = Model(inputs=[input_uer,input_movie], outputs=out2)
    model.compile(loss='mse', optimizer='Adam')
    model.summary()
    return model

In [128]:
model = Recmand_model(num_user,num_movie,100)

Tensor("input_1:0", shape=(?, ?), dtype=int32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 100)       604100      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 100)       395300      input_2[0][0]                    
______________________________________________________________

In [129]:
train_user = traindata["UserID"].values
train_movie = traindata["MovieID"].values
train_x = [train_user,train_movie]
train_y = traindata["Rating"].values
train_y = np_utils.to_categorical(train_y)

valid_user = validdata["UserID"].values
valid_movie = validdata["MovieID"].values
valid_x = [valid_user,valid_movie]
valid_y = validdata["Rating"].values
valid_y = np_utils.to_categorical(valid_y)


In [130]:
history=model.fit(train_x,train_y,batch_size = 500000,epochs =10000,callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0)],validation_data=(valid_x,valid_y))

Train on 800167 samples, validate on 100021 samples
Epoch 1/10000
800167/800167 [==============================] - 1s 1us/step - loss: 0.1389 - val_loss: 0.1388
Epoch 2/10000
800167/800167 [==============================] - 0s 0us/step - loss: 0.1388 - val_loss: 0.1387
Epoch 3/10000
800167/800167 [==============================] - 0s 0us/step - loss: 0.1387 - val_loss: 0.1387
Epoch 4/10000
800167/800167 [==============================] - 0s 0us/step - loss: 0.1387 - val_loss: 0.1386
Epoch 5/10000
800167/800167 [==============================] - 0s 0us/step - loss: 0.1386 - val_loss: 0.1385
Epoch 6/10000
800167/800167 [==============================] - 0s 0us/step - loss: 0.1385 - val_loss: 0.1384
Epoch 7/10000
800167/800167 [==============================] - 0s 0us/step - loss: 0.1384 - val_loss: 0.1384
Epoch 8/10000
800167/800167 [==============================] - 0s 0us/step - loss: 0.1384 - val_loss: 0.1383
Epoch 9/10000
800167/800167 [==============================] - 0s 0us/step -

In [131]:
def error_rate(pre,label):
    length=len(testdata)
    maxNumber=np.max(pre,axis=1)
    correct=0
    for i in range(length):
        index=np.where(pre[i]==maxNumber[i])
        index=int(index[0])
        #print(index,testdata[i])
        if index==label[i]:
            correct+=1
    print(correct)

In [132]:
pre=model.predict([testdata["UserID"].values,testdata["MovieID"].values])
maxNumber=np.max(pre,axis=1)
index=np.where(pre[0]==maxNumber[0])
index=int(index[0])
testdata.head()
label=testdata["Rating"].values
error_rate(pre,label)
len(testdata)

43684


100021

In [134]:
test_y = np_utils.to_categorical(testdata["Rating"].values)
test=model.evaluate([testdata["UserID"].values,testdata["MovieID"].values],test_y)

100021/100021 [==============================] - 2s 23us/step


In [135]:
test

0.11497638157189838

In [137]:
from keras.utils import plot_model
plot_model(model, to_file='./img/model_new.png')

In [138]:
import matplotlib.pyplot as plt


# 绘制训练 & 验证的准确率值
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig("./img/model_loss_new.png")
plt.show()

<Figure size 640x480 with 1 Axes>

In [21]:
train_user

array([5005, 4983, 4785, ..., 4868, 4907, 5530], dtype=int64)